## Test RRTMG column model with IATM=0 option

The IATM=0 option requires a different INPUT_RRTM format to IATM=1.  In this notebook, this option is tested by comparing the results with previous IATM=1 results.  On the way, useful routines will be stored in the `climatools` library.

In [310]:
%matplotlib inline
import os
import imp

import numpy as np
import pandas as pd
import xarray as xr

import climatools.rrtmg.column_model as rrtmg
import climatools.units as climaunits
import climatools.aerosol.aerosol as climaaerosol

imp.reload(rrtmg)
imp.reload(climaunits)
imp.reload(climaaerosol)

<module 'climatools.aerosol.aerosol' from '/nuwa_cluster/home/jackyu/climatools/climatools/aerosol/aerosol.py'>

In [311]:
# constants

AMD = 28.9660
AMW = 18.0160
AVOGAD = 6.02214199e23

GRAV = 9.8066

In [312]:
def column_density_dryair(h2o=None, pdp=None):
    '''
    Parameters
    ----------
    h2o: H2o volume mixing ratio
    pdp: layer pressure difference [hPa, mb]
    coldry: dry air column density [molecules/cm**2]
    '''
    dims = ('time', 'lat', 'lon', '')
    amm = (1 - h2o) * AMD + h2o * AMW
    coldry = pdp * 1e3 * AVOGAD / (1e2 * GRAV * amm * (1 + h2o))
    return coldry    

In [313]:
dir_examples = '/nuwa_cluster/home/jackyu/radiation/rrtmg/SW/examples'
dir_atm1 = 'mls_71lvls_H2O'
dir_atm0 = 'mls_71lvls_H2O_iatm0'
file_atmproh5 = 'atmpro_mls70.h5'

In [314]:
os.listdir(os.path.join(dir_examples, dir_atm1))

['rrtmg_sw.py',
 'atmpro_mls70.h5',
 'rrtmg_sw.py~',
 'INPUT_RRTM',
 'rrtmg_sw_v3.9_linux_pgi',
 'OUTPUT_RRTM',
 'tape6']

In [315]:
store = pd.HDFStore(os.path.join(dir_examples, dir_atm1, file_atmproh5))
df_atmpro = store['atmpro']
store.close()

df_atmpro = df_atmpro.reset_index().drop('index', axis=1)
df_atmpro.index.names = ['ilev']
df_atmpro.head()

,altitude,pressure,temperature,H2O,CO2,O3,N2O,CO,CH4,O2
ilev,,,,,,,,,,
0,89.60800,0.00242,212.06955,6.4321,0,0,0,0,0,0
1,87.50455,0.00339,212.61195,6.4321,0,0,0,0,0,0
2,85.39563,0.00476,213.15535,6.4321,0,0,0,0,0,0
3,83.28138,0.00667,213.69980,6.4321,0,0,0,0,0,0
4,81.16173,0.00936,214.24530,6.4321,0,0,0,0,0,0


In [316]:
# convert pandas.DataFrame to xarray.Dataset
ds = xr.Dataset.from_dataframe(df_atmpro)    

In [317]:
# rename the data arrays
ds = ds.rename({'pressure': 'level_pressure',
                'altitude': 'level_altitude',
                'temperature': 'level_temperature',
                'H2O': 'level_ppmv_h2o',
                'CO2': 'level_ppmv_co2',
                'O3': 'level_ppmv_o3',
                'N2O': 'level_ppmv_n2o',
                'CO': 'level_ppmv_co',
                'CH4': 'level_ppmv_ch4',
                'O2': 'level_ppmv_o2'})

In [318]:
# compute layer values for all variables
for name in ds.data_vars:
    layer_data = .5 * (ds[name][:-1].values + ds[name][1:].values)
    ds[name.replace('level', 'layer')] = (('lev',), layer_data)

In [319]:
# increase layer and level index numbers by 1
ds.coords['lev'] = range(1, ds.dims['lev'] + 1)
ds.coords['ilev'] = range(1, ds.dims['ilev'] + 1)

In [320]:
# reindex layer and level with an additional index number
ds = ds.reindex(lev=range(ds.dims['lev'] + 1))
ds = ds.reindex(ilev=range(ds.dims['ilev'] + 1))

In [321]:
# assign top layer and leve pressures
ds['layer_pressure'][dict(lev=0)] = (.5 * ds['level_pressure']
                                     .isel(ilev=1))
ds['level_pressure'][dict(ilev=0)] = 1e-4

In [322]:
# assign top layer and level temperatures
ds['layer_temperature'][dict(lev=0)] = (ds['layer_temperature']
                                        [dict(lev=1)])
ds['level_temperature'][dict(ilev=1)] = (.5 * 
                                         sum(ds['layer_temperature']
                                             [dict(lev=[0, 1])]))
ds['level_temperature'][dict(ilev=0)] = (ds['level_temperature']
                                         [dict(ilev=1)])

In [323]:
# assign top layer molecule densities
names_molecules = ['h2o', 'co2', 'o3', 'n2o', 'co', 'ch4', 'o2']
for molecule in names_molecules:
    name_var = 'layer_ppmv_' + molecule
    ds[name_var][dict(lev=0)] = ds[name_var][dict(lev=1)]

In [324]:
# set layer and lever pressure as coordinate variables
ds = ds.set_coords(['layer_pressure', 'level_pressure'])

In [325]:
# compute layer pressure difference
ds = climaaerosol.get_pressure_difference(ds)

In [326]:
# convert molecules densities from ppmv to vmr
names_molecules = ['h2o', 'co2', 'o3', 'n2o', 'co', 'ch4', 'o2']
for molecule in names_molecules:
    name_ppmv = 'layer_ppmv_' + molecule
    name_vmr = name_ppmv.replace('ppmv', 'vmr')
    ds[name_vmr] = 1e-6 * ds[name_ppmv]


In [327]:
# drop data variables for molecule densities in ppmv
ds = ds.drop([name for name in ds.data_vars if '_ppmv_' in name])

In [328]:
# calculate column density of dry air 
coldens_dryair = column_density_dryair(h2o=ds['layer_vmr_h2o'],
                                       pdp=ds['dpressure'])
ds['layer_coldens_dryair'] = (('lev',), coldens_dryair)

h2o_lev1 = ds['layer_vmr_h2o'][dict(lev=1)]
amm = (1 - h2o_lev1) * AMD + h2o_lev1 * AMW
ds['layer_coldens_dryair'][dict(lev=0)] = (ds['level_pressure'][dict(ilev=1)] 
                                           * 1e3 * AVOGAD / 
                                           (1e2 * GRAV * amm * (1 + h2o_lev1)))

In [329]:
# convert molecule densities from vmr to column density
names_molecules = ['h2o', 'co2', 'o3', 'n2o', 'co', 'ch4', 'o2']
for name in names_molecules:
    ds['layer_coldens_' + name] = (ds['layer_vmr_' + name] 
                                   * ds['layer_coldens_dryair'])

In [330]:
# drop data variables for molecule densities in vmr
ds = ds.drop([name for name in ds.data_vars if '_vmr_' in name])

In [331]:
ds['juldat'] = ((), 0., {'long_name': 'Julian day associated with calculation', 
                         'units': 'days'})

ds['sza'] = ((), 60, {'long_name': 'solar zenith angle in degrees',
                      'units': 'degrees'})

In [332]:
ds['layer_coldens_h2o']

<xarray.DataArray 'layer_coldens_h2o' (lev: 71)>
array([  3.29997325e+14,   1.32271655e+14,   1.86816668e+14,
         2.60452434e+14,   3.66815209e+14,   5.14086742e+14,
         7.19994164e+14,   1.01180998e+15,   1.41817032e+15,
         1.98952933e+15,   2.79134101e+15,   3.91496827e+15,
         5.49131913e+15,   7.70311938e+15,   1.08053670e+16,
         1.51498772e+16,   2.12725549e+16,   2.98088493e+16,
         4.18360245e+16,   5.86767971e+16,   8.23084238e+16,
         1.15444519e+17,   1.61944142e+17,   2.27166341e+17,
         3.18638326e+17,   4.46969104e+17,   6.26954009e+17,
         8.79442872e+17,   1.23360364e+18,   1.73038597e+18,
         2.42723942e+18,   3.24201918e+18,   3.24201918e+18,
         3.24262402e+18,   3.27767968e+18,   3.79698699e+18,
         6.88564962e+18,   1.60044486e+19,   3.46537505e+19,
         6.70369938e+19,   1.11168292e+20,   1.58058625e+20,
         2.06482594e+20,   2.61970464e+20,   3.27211261e+20,
         4.03086508e+20,   4.8752868

In [333]:
os.chdir(os.path.join(dir_examples, dir_atm0))

rrtmg.write_input_rrtm(ds=ds, aerosol=False, iatm=0)

In [307]:
1

1